In [ ]:
from fastai.gen_doc.gen_notebooks import update_module_metadata
import fastai.callbacks.fp16
# For updating jekyll metadata. You MUST reload notebook immediately after executing this cell for changes to save
# Leave blank to autopopulate from mod.__doc__
update_module_metadata(fastai.callbacks.fp16, title='callback.fp16', summary='Training in mixed precision implementation')

# Mixed precision training

This module allows the forward and backward passes of your neural net to be done in fp16 (also known as *half precision*). This is particularly important if you have an NVIDIA GPU with [tensor cores](https://www.nvidia.com/en-us/data-center/tensorcore/), since it can speed up your training by 200% or more.

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.callbacks.fp16 import * 
from fastai.docs import *
from fastai import *
from fastai.train import *

## Overview

To train your model in mixed precision you just have to call `Learner.to_fp16`, which converts the model and modifies the existing `Learner` to add `MixedPrecision`.

In [ ]:
show_doc(Learner.to_fp16)

#### <a id=to_fp16></a>`to_fp16`
> `to_fp16`(`learn`:`Learner`, `loss_scale`:`float`=`512.0`, `flat_master`:`bool`=`False`) -> `Learner`


Transforms the learner in FP16 precision <a href="https://github.com/fastai/fastai/blob/master/fastai/train.py#L26">[source]</a>

For example:

In [ ]:
learn = ConvLearner(get_mnist(), tvm.resnet18, metrics=accuracy).to_fp16()
learn.fit_one_cycle(1)

Total time: 00:05
epoch  train loss  valid loss  accuracy
0      0.104780    0.063669    0.977429  (00:05)



Details about mixed precision training are available in [NVIDIA's documentation](https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html). We will just summarize the basics here.

The only parameter you may want to tweak is `loss_scale`. This is used to scale the loss up, so that it doesn't underflow fp16, leading to loss of accuracy (this is reversed for the final gradient calculation after converting back to fp32). Generally the default `512` works well, however. You can also enable or disable the flattening of the master parameter tensor with `flat_master=True`, however in our testing the different is negligible.

Internally, the callback ensures that all model parameters (except batchnorm layers, which require fp32) are converted to fp16, and an fp32 copy is also saved. The fp32 copy (the `master` parameters) is what is used for actually updating with the optimizer; the fp16 parameters are used for calculating gradients. This helps avoid underflow with small learning rates.

All of this is implemented by the following Callback.

In [ ]:
show_doc(MixedPrecision)

## <a id=MixedPrecision></a>`class` `MixedPrecision`
> `MixedPrecision`(`learn`:[`Learner`](/basic_train.html#Learner), `loss_scale`:`float`=`512.0`, `flat_master`:`bool`=`False`) :: [`Callback`](/callback.html#Callback)


Callback that handles mixed-precision training. <a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L56">[source]</a>

You don't have to call the following functions yourself - they're called by the callback framework automatically. They're just documented here so you can see exactly what the callback is doing.

In [ ]:
show_doc(MixedPrecision.on_backward_begin)

#### <a id=on_backward_begin></a>`on_backward_begin`
> `on_backward_begin`(`last_loss`:`Rank0Tensor`, `kwargs`:`Any`) -> `Rank0Tensor`


Scale gradients up by `loss_scale` to prevent underflow <a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L90">[source]</a>

In [ ]:
show_doc(MixedPrecision.on_backward_end)

#### <a id=on_backward_end></a>`on_backward_end`
> `on_backward_end`(`kwargs`:`Any`)


Convert the gradients back to FP32 and divide them by the scale. <a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L95">[source]</a>

In [ ]:
show_doc(MixedPrecision.on_loss_begin)

#### <a id=on_loss_begin></a>`on_loss_begin`
> `on_loss_begin`(`last_output`:`Tensor`, `kwargs`:`Any`) -> `Tensor`


Converts half precision output to FP32 to avoid reduction overflow. <a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L86">[source]</a>

In [ ]:
show_doc(MixedPrecision.on_step_end)

#### <a id=on_step_end></a>`on_step_end`
> `on_step_end`(`kwargs`:`Any`)


Update the params from master to model and zero grad <a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L101">[source]</a>

In [ ]:
show_doc(MixedPrecision.on_train_begin)

#### <a id=on_train_begin></a>`on_train_begin`
> `on_train_begin`(`kwargs`:`Any`)


Ensures everything is in half precision mode <a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L63">[source]</a>

In [ ]:
show_doc(MixedPrecision.on_train_end)

#### <a id=on_train_end></a>`on_train_end`
> `on_train_end`(`kwargs`:`Any`)


Removes half precision transforms added at `on_train_begin` <a href="https://github.com/fastai/fastai/blob/master/fastai/callbacks/fp16.py#L80">[source]</a>